In [1]:
from Bio import Entrez, SeqIO
import pandas as pd
import numpy as np
import os

In [3]:
# Informing NCBI
Entrez.email = "hedwardes.ai@gmail.com"


In [4]:
# Search parameters
organism = "Mus musculus[Organism]"
gene = "Trp53[Gene]"
search_term = f"{organism} AND {gene}"

# Perform the search
handle = Entrez.esearch(db="nucleotide", term=search_term, retmax=100)
record = Entrez.read(handle)
handle.close()

# Get a list of GenBank IDs
genbank_ids = record["IdList"]
print(f"Found {len(genbank_ids)} records for {gene} in {organism}.")

Found 13 records for Trp53[Gene] in Mus musculus[Organism].


In [5]:
# Directory to save sequences
os.makedirs('sequences', exist_ok=True)

sequences = []

for genbank_id in genbank_ids:
    try:
        handle = Entrez.efetch(db="nucleotide", id=genbank_id, rettype="gb", retmode="text")
        seq_record = SeqIO.read(handle, "genbank")
        handle.close()
        sequences.append(seq_record)
        # Save each sequence to a FASTA file
        SeqIO.write(seq_record, f"sequences/{seq_record.id}.fasta", "fasta")
        print(f"Retrieved and saved: {seq_record.id}, Length: {len(seq_record.seq)}")
    except Exception as e:
        print(f"An error occurred while retrieving {genbank_id}: {e}")

Retrieved and saved: NM_001127233.2, Length: 1824
Retrieved and saved: NM_011640.4, Length: 1728
Retrieved and saved: XM_030245923.2, Length: 2031
Retrieved and saved: XM_006533157.5, Length: 2136
Retrieved and saved: NC_000077.7, Length: 121973369
Retrieved and saved: XM_030245922.1, Length: 1881
Retrieved and saved: CH466601.1, Length: 7647059
Retrieved and saved: AF367373.1, Length: 1502
Retrieved and saved: CM000219.2, Length: 132665419
Retrieved and saved: BC005448.1, Length: 1782
Retrieved and saved: AY044188.1, Length: 1176
Retrieved and saved: AL731687.13, Length: 116898
Retrieved and saved: EU031806.1, Length: 1250


In [10]:
# Pre-Processing

# Filter sequences based on length and content
min_length = 500  # Define minimum acceptable sequence length
clean_sequences = []

for seq_record in sequences:
    sequence = str(seq_record.seq).upper()
    if len(sequence) >= min_length and 'N' not in sequence:
        clean_sequences.append(seq_record)
    else:
        print(f"Sequence {seq_record.id} excluded due to insufficient length or ambiguous nucleotides.")

print(f"Number of clean sequences: {len(clean_sequences)}")

Sequence NC_000077.7 excluded due to insufficient length or ambiguous nucleotides.
Sequence CH466601.1 excluded due to insufficient length or ambiguous nucleotides.
Sequence CM000219.2 excluded due to insufficient length or ambiguous nucleotides.
Number of clean sequences: 10


In [12]:
# Extract Coding Sequences (CDS) as mutations in the coding regions are more likely to affect gene function.

# Create a dictionary mapping sequence IDs to their sequences
references = {record.id: record.seq for record in clean_sequences}

cds_sequences = []

for seq_record in clean_sequences:
    for feature in seq_record.features:
        if feature.type == "CDS":
            try:
                # Provide the references dictionary to handle referenced sequences
                cds_seq = feature.extract(seq_record.seq, references=references)
                cds_sequences.append({
                    'id': seq_record.id,
                    'sequence': str(cds_seq),
                    'description': seq_record.description
                })
            except ValueError as e:
                print(f"Skipping feature in {seq_record.id} due to error: {e}")

print(f"Number of CDS sequences extracted: {len(cds_sequences)}")


Skipping feature in AL731687.13 due to error: Feature references another sequence (AL596125.27), not found in references
Skipping feature in AL731687.13 due to error: Feature references another sequence (AL603707.5), not found in references
Number of CDS sequences extracted: 17


In [13]:
# Convert to DataFrame
df_sequences = pd.DataFrame(cds_sequences)
print(df_sequences.head())

               id                                           sequence  \
0  NM_001127233.2  ATGACTGCCATGGAGGAGTCACAGTCGGATATCAGCCTCGAGCTCC...   
1     NM_011640.4  ATGACTGCCATGGAGGAGTCACAGTCGGATATCAGCCTCGAGCTCC...   
2  XM_030245923.2  ATGACTGCCATGGAGGAGTCACAGTCGGATATCAGCCTCGAGCTCC...   
3  XM_006533157.5  ATGACTGCCATGGAGGAGTCACAGTCGGATATCAGCCTCGAGCTCC...   
4  XM_030245922.1  ATGACTGCCATGGAGGAGTCACAGTCGGATATCAGCCTCGAGCTCC...   

                                         description  
0  Mus musculus transformation related protein 53...  
1  Mus musculus transformation related protein 53...  
2  PREDICTED: Mus musculus transformation related...  
3  PREDICTED: Mus musculus transformation related...  
4  PREDICTED: Mus musculus transformation related...  


In [14]:
def assign_label(description):
    description_lower = description.lower()
    if "mutant" in description_lower or "mutation" in description_lower or "variant" in description_lower:
        return 1  # Mutated
    else:
        return 0  # Normal

df_sequences['label'] = df_sequences['description'].apply(assign_label)

# Check label distribution
print(df_sequences['label'].value_counts())

label
0    12
1     5
Name: count, dtype: int64
